<a href="https://colab.research.google.com/github/nikitawokurka/Winemodel/blob/master/wine_quality_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Cloning of the wine dataset from a github repository.

In [0]:
! git clone https://github.com/nikitawokurka/Winemodel.git
%cd Winemodel 
%ls

Cloning into 'Winemodel'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
/content/Winemodel/Winemodel/Winemodel
README.md  wine_quality_model.ipynb  winequality-red.csv


# Import TensorFlow and other libraries

Installing Tensorflow 2.0 beta via pip.



In [0]:
pip install tensorflow==2.0.0-beta0

Importing libraries and checking the version of Tensorflow

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pandas as pd
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

print(tf.__version__)

2.0.0-beta0


# Use Pandas to create a dataframe

Reading the winequality-red.csv  dataset and displaying the head.

In [0]:
dataframe = pd.read_csv("winequality-red.csv", dtype=float, delimiter=',')
dataframe.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5.0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5.0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6.0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5.0


Splitting the dataset into a test and validation set.

In [0]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

1023 train examples
256 validation examples
320 test examples


# Create an input pipeline using tf.data



In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('quality')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [0]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

# Create a feature layer

In [0]:
#Definig feature_colums
feature_columns = []

# numeric cols
for header in ["fixed_acidity", "volatile_acidity", "citric_acid", "residual_sugar", "chlorides", "free_sulfur_dioxide", "total_sulfur_dioxide", "density", "pH", "sulphates", "alcohol"]:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)


# Create, compile, and train the mode

Setting the amount of nodes for each layer. Output layer has 10 nodes for the quality betweenn 0 and 10.

Quote from wineDataesciption -
"quality 
between 0 (very bad) and 10 (very excellent)"



In [0]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(64, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(10, activation='softmax')
])

Using the spasparse categorical crossentropy, because we have 10 label classes.

linke to API from [SparseCategoricalCrossentropy](hhttps://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy)

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Using only 10 epochs to not overfit our model.



In [0]:
print('Train data set')
model.fit(train_ds,
          validation_data=val_ds,
          epochs=10)

Train data set
Epoch 1/10
32/32 [==============================] - 2s 49ms/step - loss: 2.1360 - accuracy: 0.2880 - val_loss: 1.3579 - val_accuracy: 0.5078
Epoch 2/10
32/32 [==============================] - 0s 6ms/step - loss: 1.2644 - accuracy: 0.4525 - val_loss: 1.2341 - val_accuracy: 0.4648
Epoch 3/10
32/32 [==============================] - 0s 6ms/step - loss: 1.1662 - accuracy: 0.4494 - val_loss: 1.1622 - val_accuracy: 0.5117
Epoch 4/10
32/32 [==============================] - 0s 6ms/step - loss: 1.1491 - accuracy: 0.4722 - val_loss: 1.1478 - val_accuracy: 0.5273
Epoch 5/10
32/32 [==============================] - 0s 6ms/step - loss: 1.1394 - accuracy: 0.4702 - val_loss: 1.1460 - val_accuracy: 0.5156
Epoch 6/10
32/32 [==============================] - 0s 5ms/step - loss: 1.1333 - accuracy: 0.4895 - val_loss: 1.1332 - val_accuracy: 0.5195
Epoch 7/10
32/32 [==============================] - 0s 6ms/step - loss: 1.1299 - accuracy: 0.4720 - val_loss: 1.1355 - val_accuracy: 0.5352
Epoc

Testing model on test data set.

In [0]:
print('Test data set')
loss, accuracy = model.evaluate(test_ds)
print("Validation accuracy", accuracy)

Test data set
10/10 [==============================] - 0s 4ms/step - loss: 1.2100 - accuracy: 0.5469
Validation accuracy 0.546875


# Overfitting

Using 200 epochs to see that the accuracy between the train data set and the test data set differ, wich is an indicator that the model is overfitting on the train data set. 
(val_accuracy = accuracy form the model on the test data set)

In [0]:
print('Train data set')
model.fit(train_ds,
          validation_data=val_ds,
          epochs=200)

print('Test data set')
loss, accuracy = model.evaluate(test_ds)
print("Validation accuracy", accuracy)

Train data set
Epoch 1/200
32/32 [==============================] - 0s 6ms/step - loss: 1.1137 - accuracy: 0.5122 - val_loss: 1.1233 - val_accuracy: 0.5273
Epoch 2/200
32/32 [==============================] - 0s 6ms/step - loss: 1.1086 - accuracy: 0.5132 - val_loss: 1.1133 - val_accuracy: 0.5391
Epoch 3/200
32/32 [==============================] - 0s 6ms/step - loss: 1.1055 - accuracy: 0.5269 - val_loss: 1.1124 - val_accuracy: 0.5312
Epoch 4/200
32/32 [==============================] - 0s 6ms/step - loss: 1.0994 - accuracy: 0.5200 - val_loss: 1.1085 - val_accuracy: 0.5117
Epoch 5/200
32/32 [==============================] - 0s 5ms/step - loss: 1.0953 - accuracy: 0.5181 - val_loss: 1.1039 - val_accuracy: 0.5391
Epoch 6/200
32/32 [==============================] - 0s 5ms/step - loss: 1.0915 - accuracy: 0.5249 - val_loss: 1.0980 - val_accuracy: 0.5625
Epoch 7/200
32/32 [==============================] - 0s 5ms/step - loss: 1.0884 - accuracy: 0.5288 - val_loss: 1.0921 - val_accuracy: 0.550